In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from comtrade import Comtrade
import seaborn as sns

##### Functions to be used

In [ ]:
def data_frames_creation(paths):
    temp=[]
    for i in range (0, len(paths)):
        rec = Comtrade()
        rec.load(paths[i][0],paths[i][1])
        samples_list=[]
        for j in range (0, len(rec.analog)):
            samples_list.append(rec.analog[j])
        abc=pd.DataFrame(samples_list)
        abc=abc.transpose()
        abc.columns=rec.analog_channel_ids
        time=pd.Series(data=rec.time,name='Time')
        abc=pd.concat([time,abc],axis=1)
        temp.append(abc)
        samples_list=[]
    return temp

In [ ]:
import os
def getPath(file_id, file_type):
    response = ''
    directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\DAT_Files"
    if file_type == 2:
        directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\CFG_FILES"
    for path in os.listdir(directory):
        full_path = os.path.join(directory, path)
        if os.path.isfile(full_path):
            if str(file_id) in full_path:
                response = full_path
    return response

In [ ]:
def single_point_visula(cols,data):
    plt.figure(figsize=(11,2))
    plt.title('Currents Amplitude Vs Time of '+cols)
    plt.plot(data['Time'],data[cols],color='#0FE279')
    plt.xlabel('Time', fontsize=8)
    plt.ylabel('Current (amp)', fontsize=7)

In [ ]:
def box_plots(data,b):
    plt.figure(figsize=(11,3))
    plt.title('Distribution of Data Point '+ str (b))
    sns.boxplot(data=data,color='#B0FC21')
    plt.xlabel('Phases', fontsize=9)
    plt.ylabel('Current (amp)', fontsize=10)

In [ ]:
def pred_fal_good_Visualization(dataframe, wrong_phases,columns):
    for f in range (0,len(dataframe)):
        if wrong_phases[f]=='A':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[0])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I A'],color='#E9967A')
        if wrong_phases[f]=='B':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[1])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I B'],color='#108A0D')
        if wrong_phases[f]=='C':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[2])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I C'],color='#0282B9')
        if wrong_phases[f]=='N':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[3])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:IN'],color='#C794F2')

###### `********************************************************************************************************************`

##### Data Loading

In [ ]:
ten_PCA=pd.read_csv('Ten_com_clustering.csv')

In [ ]:
ten_PCA

In [ ]:
ten_PCA.isna().sum()

##### Shuffling and Splits

In [ ]:
np.random.seed(42)
ten_PCA_shuffle=ten_PCA.sample(frac=1)

In [ ]:
ten_PCA_shuffle

In [ ]:
engin_df=pd.read_csv('eng_features.csv')
engin_df.head()

In [ ]:
np.random.seed(42)
engin_df_shuffle=engin_df.sample(frac=1)
engin_df_shuffle

In [ ]:
train_set_len=round(len(ten_PCA_shuffle)*0.8)

In [ ]:
test_set_len=round(len(ten_PCA_shuffle)*0.2)

In [ ]:
print(train_set_len)
print(test_set_len)

In [ ]:
train_set=ten_PCA_shuffle.iloc[0:1098,:]

In [ ]:
test_set=ten_PCA_shuffle.iloc[1098:,:]

In [ ]:
test_set

In [ ]:
train_set_engin_df=engin_df_shuffle.iloc[0:1098,:]

In [ ]:
test_set_engin_df=engin_df_shuffle.iloc[1098:,:]

In [ ]:
train_set_engin_df

In [ ]:
test_set_engin_df

#### GMM-Diagonal

In [ ]:
train_set

In [ ]:
random_states=[1,23,5,66,9]

In [ ]:
for i in random_states:
    from sklearn.mixture import GaussianMixture
    gmm_diag_train = GaussianMixture(n_components=2,covariance_type='diag',random_states=i).fit(train_set)

In [ ]:
gmm_diag_train.get_params()

In [ ]:
np.random.seed(40)
gmm_dia_train_labels=gmm_diag_train.predict(train_set)

In [ ]:
gmm_dia_train_labels

In [ ]:
len(gmm_dia_train_labels)

In [ ]:
gmm_train_lab_df=pd.DataFrame(gmm_dia_train_labels,columns=['Train_labels'])

In [ ]:
gmm_train_lab_df

In [ ]:
train_set_reset=train_set.reset_index(drop=True)
train_set_reset

In [ ]:
from sklearn.metrics import silhouette_score
print('Silhoutte score of GMM clustering is ' + str(silhouette_score(train_set, gmm_dia_train_labels)))

In [ ]:
gmm_dia_train_df = pd.concat([train_set_reset, gmm_train_lab_df], axis=1)

In [ ]:
gmm_dia_train_df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
gmm_dia_train_df['Train_labels'].value_counts()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(gmm_dia_train_df.iloc[:,0],gmm_dia_train_df.iloc[:,1], hue=gmm_dia_train_df['Train_labels'], palette='magma', s=70, alpha=0.8).set_title('Distribution of cluster Based on Gaussian Mixture Model (Train set)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

## Predicted Failure Verification by GMM Diagonal (Train Set)

In [ ]:
gmm_dia_train_df

In [ ]:
pred_fal_gmm_dia_train=gmm_dia_train_df[gmm_dia_train_df['Train_labels']==1]

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia_train.iloc[:,0],pred_fal_gmm_dia_train.iloc[:,1],color='#08BF4B' , s=70, alpha=0.8).set_title('Only Predicted Failure cluster by GMM on Train set',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only (Train Set)'])
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
pred_fal_gmm_dia_train

In [ ]:
predicted_failure_gmm_index_train=pred_fal_gmm_dia_train.index

In [ ]:
predicted_failure_gmm_index_train

In [ ]:
train_set_engin_df_reset=train_set_engin_df.reset_index(drop=True)
train_set_engin_df_reset

In [ ]:
Faulty_pred_gmm_ids_train=[]
for i in predicted_failure_gmm_index_train:
    Faulty_pred_gmm_ids_train.append(train_set_engin_df_reset.loc[i])
Faulty_pred_gmm_ids_train=pd.DataFrame(Faulty_pred_gmm_ids_train)

In [ ]:
Faulty_pred_gmm_ids_train

In [ ]:
len(Faulty_pred_gmm_ids_train)

In [ ]:
gmm_dia_failure_paths_train=[]
for index, row in Faulty_pred_gmm_ids_train.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_failure_paths_train.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_failure_paths_train

In [ ]:
gmm_dia_faulty_phases_train=[]
for index, row in Faulty_pred_gmm_ids_train.iterrows():
    gmm_dia_faulty_phases_train.append(row['Phase'])

In [ ]:
print(len(gmm_dia_failure_paths_train))
print(len(gmm_dia_faulty_phases_train))

In [ ]:
gmm_dia_fal_traindata=data_frames_creation(gmm_dia_failure_paths_train)

In [ ]:
col_names=[ 'MPI3p1:I A', 'MPI3p1:I B', 'MPI3p1:I C', 'MPI3p1:IN']

In [ ]:
pred_fal_good_Visualization(gmm_dia_fal_traindata,gmm_dia_faulty_phases_train,col_names)

In [ ]:
false_failure_trainset=[3,6,145,174,196,212,280]

In [ ]:
len(false_failure_trainset)

In [ ]:
pred_fal_gmm_dia_train_reset=pred_fal_gmm_dia_train.reset_index(drop=True)

In [ ]:
false_neg_train=pred_fal_gmm_dia_train_reset.iloc[false_failure_trainset]
false_neg_train

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia_train.iloc[:,0],pred_fal_gmm_dia_train.iloc[:,1],color='#B2FF33', s=70, alpha=0.8).set_title('Predicted Failure cluster based on GMM (Train set)',fontsize=13)
sns.scatterplot(false_neg_train.iloc[:,0],false_neg_train.iloc[:,1], color='#FF6E33' ,s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
#plt.savefig('failure_only')
plt.show()

## Predicted Good Verification (Trian Set)

In [ ]:
pred_good_gmm_dia_train=gmm_dia_train_df[gmm_dia_train_df['Train_labels']==0]

In [ ]:
pred_good_gmm_dia_train.head()

In [ ]:
len(pred_good_gmm_dia_train)

In [ ]:
plt.figure(figsize = (17,7))
sns.scatterplot(pred_good_gmm_dia_train.iloc[:,0],pred_good_gmm_dia_train.iloc[:,1], color='#D133FF',  s=70, alpha=0.8).set_title(' Only Predicted Good cluster based GMM (Train Set)',fontsize=13)
plt.legend(labels=['Predicted Good Cluster Only'],loc='upper left')
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
pred_good_gmm_dia_train_index=pred_good_gmm_dia_train.index
pred_good_gmm_dia_train_index

In [ ]:
len(pred_good_gmm_dia_train_index)

In [ ]:
train_set_engin_df_reset

In [ ]:
pred_good_gmm_dia_train_ids=[]
for i in pred_good_gmm_dia_train_index:
    pred_good_gmm_dia_train_ids.append(train_set_engin_df_reset.loc[i])
pred_good_gmm_dia_train_ids=pd.DataFrame(pred_good_gmm_dia_train_ids)

In [ ]:
len(pred_good_gmm_dia_train_ids)

In [ ]:
pred_good_train_paths=[]
for index, row in pred_good_gmm_dia_train_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    pred_good_train_paths.append([CFG_Path, dAT_path])

In [ ]:
pred_good_train_phases=[]
for index, row in pred_good_gmm_dia_train_ids.iterrows():
    pred_good_train_phases.append(row['Phase'])

In [ ]:
len(pred_good_train_phases)

In [ ]:
pred_good_datasets=data_frames_creation(pred_good_train_paths)

In [ ]:
len(pred_good_datasets)

In [ ]:
pred_fal_good_Visualization(pred_good_datasets,pred_good_train_phases,col_names)

In [ ]:
false_predgood_train=[705]

In [ ]:
pred_good_gmm_dia_train_reset=pred_good_gmm_dia_train.reset_index(drop=True)

In [ ]:
false_predgood_trainset_=pred_good_gmm_dia_train_reset.iloc[false_predgood_train]

In [ ]:
len(false_predgood_trainset_)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_good_gmm_dia_train.iloc[:,0],pred_good_gmm_dia_train.iloc[:,1], color='#B2FF33', s=70, alpha=0.8).set_title(' True Positive Vs False Positive Baes on GMM (Train set)',fontsize=13)
sns.scatterplot(false_predgood_trainset_.iloc[:,0],false_predgood_trainset_.iloc[:,1], color='#FF6E33', s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Positive","False Positive"])
#plt.savefig('good_pred')
plt.show()

#### Prediction on Test set

In [ ]:
np.random.seed(40)
gmm_dia_test_labels=gmm_diag_train.predict(test_set)

In [ ]:
len(gmm_dia_test_labels)

In [ ]:
gmm_test_lab_df=pd.DataFrame(gmm_dia_test_labels,columns=['Test_labels'])

In [ ]:
gmm_test_lab_df['Test_labels'].value_counts()

In [ ]:
test_set_reset=test_set.reset_index(drop=True)
test_set_reset

In [ ]:
print('Silhoutte score of GMM clustering (Test set) is ' + str(silhouette_score(test_set, gmm_dia_test_labels)))

In [ ]:
gmm_dia_test_df = pd.concat([test_set_reset, gmm_test_lab_df], axis=1)

In [ ]:
plt.figure(figsize = (17,7))
sns.scatterplot(gmm_dia_test_df.iloc[:,0],gmm_dia_test_df.iloc[:,1], hue=gmm_dia_test_df['Test_labels'], palette='magma', s=70, alpha=0.8).set_title('Distribution of cluster Based on Gaussian Mixture Model (Test set)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

## Predicted Failure Verification by GMM Diagonal (Test Set)

In [ ]:
pred_fal_gmm_dia_test=gmm_dia_test_df[gmm_dia_test_df['Test_labels']==1]

In [ ]:
pred_fal_gmm_dia_test

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia_test.iloc[:,0],pred_fal_gmm_dia_test.iloc[:,1],color='#08BF4B' , s=70, alpha=0.8).set_title('Only Predicted Failure cluster by GMM on Test set',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only (Test Set)'])
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
test_set_engin_df_reset=test_set_engin_df.reset_index(drop=True)
test_set_engin_df_reset

In [ ]:
predicted_failure_gmm_index_test=pred_fal_gmm_dia_test.index

In [ ]:
predicted_failure_gmm_index_test

In [ ]:
Faulty_pred_gmm_ids_test=[]
for i in predicted_failure_gmm_index_test:
    Faulty_pred_gmm_ids_test.append(test_set_engin_df_reset.loc[i])
Faulty_pred_gmm_ids_test=pd.DataFrame(Faulty_pred_gmm_ids_test)

In [ ]:
Faulty_pred_gmm_ids_test

In [ ]:
gmm_dia_failure_paths_test=[]
for index, row in Faulty_pred_gmm_ids_test.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_failure_paths_test.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_faulty_phases_test=[]
for index, row in Faulty_pred_gmm_ids_test.iterrows():
    gmm_dia_faulty_phases_test.append(row['Phase'])

In [ ]:
gmm_dia_fal_testdata=data_frames_creation(gmm_dia_failure_paths_test)

In [ ]:
pred_fal_good_Visualization(gmm_dia_fal_testdata,gmm_dia_faulty_phases_test,col_names)

In [ ]:
false_failure_testset=[12,65]

In [ ]:
pred_fal_gmm_dia_test

In [ ]:
pred_fal_gmm_dia_test_reset=pred_fal_gmm_dia_test.reset_index(drop=True)

In [ ]:
false_neg_test=pred_fal_gmm_dia_test_reset.iloc[false_failure_testset]
false_neg_test

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia_test.iloc[:,0],pred_fal_gmm_dia_test.iloc[:,1],color='#B2FF33', s=70, alpha=0.8).set_title('Predicted Failure cluster based on GMM (Test set)',fontsize=13)
sns.scatterplot(false_neg_test.iloc[:,0],false_neg_test.iloc[:,1], color='#FF6E33' ,s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
#plt.savefig('failure_only')
plt.show()

## Predicted Good Verification (Test Set)

In [ ]:
pred_good_gmm_dia_test=gmm_dia_test_df[gmm_dia_test_df['Test_labels']==0]

In [ ]:
plt.figure(figsize = (17,7))
sns.scatterplot(pred_good_gmm_dia_test.iloc[:,0],pred_good_gmm_dia_test.iloc[:,1], color='#691381',  s=70, alpha=0.8).set_title(' Only Predicted Good cluster based GMM (Test Set)',fontsize=13)
plt.legend(labels=['Predicted Good Cluster Only'],loc='upper left')
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
pred_good_gmm_dia_test_index=pred_good_gmm_dia_test.index
pred_good_gmm_dia_test_index

In [ ]:
pred_good_gmm_dia_test_ids=[]
for i in pred_good_gmm_dia_test_index:
    pred_good_gmm_dia_test_ids.append(test_set_engin_df_reset.loc[i])
pred_good_gmm_dia_test_ids=pd.DataFrame(pred_good_gmm_dia_test_ids)

In [ ]:
pred_good_test_paths=[]
for index, row in pred_good_gmm_dia_test_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    pred_good_test_paths.append([CFG_Path, dAT_path])

In [ ]:
pred_good_test_phases=[]
for index, row in pred_good_gmm_dia_test_ids.iterrows():
    pred_good_test_phases.append(row['Phase'])

In [ ]:
pred_goodtest_datasets=data_frames_creation(pred_good_test_paths)

In [ ]:
pred_fal_good_Visualization(pred_goodtest_datasets,pred_good_test_phases,col_names)

In [ ]:
false_predgood_test=[112]

In [ ]:
pred_good_gmm_dia_test_reset=pred_good_gmm_dia_test.reset_index(drop=True)
false_predgood_testset_=pred_good_gmm_dia_test_reset.iloc[false_predgood_test]

In [ ]:
false_predgood_testset_

In [ ]:
plt.figure(figsize = (17,7))
sns.scatterplot(pred_good_gmm_dia_test.iloc[:,0],pred_good_gmm_dia_test.iloc[:,1], color='#B2FF33', s=70, alpha=0.8).set_title(' True Positive Vs False Positive Baesd on GMM (Test set)',fontsize=13)
sns.scatterplot(false_predgood_testset_.iloc[:,0],false_predgood_testset_.iloc[:,1], color='#FF6E33', s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Positive","False Positive"])
#plt.savefig('good_pred')
plt.show()

#### Evaluation Metrics on Train and test Set

###### Train Sets

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

In [ ]:
Silhouette_score_trainset=silhouette_score(train_set,gmm_dia_train_labels)
calinski_harabasz_score_trainset=calinski_harabasz_score(train_set,gmm_dia_train_labels)
davies_bouldin_score_trainset=davies_bouldin_score(train_set,gmm_dia_train_labels)

In [ ]:
print(Silhouette_score_trainset)
print(calinski_harabasz_score_trainset)
print(davies_bouldin_score_trainset)

###### Test Sets

In [ ]:
Silhouette_score_testset=silhouette_score(test_set,gmm_dia_test_labels)
calinski_harabasz_score_testset=calinski_harabasz_score(test_set,gmm_dia_test_labels)
davies_bouldin_score_testset=davies_bouldin_score(test_set,gmm_dia_test_labels)

In [ ]:
print(Silhouette_score_testset)
print(calinski_harabasz_score_testset)
print(davies_bouldin_score_testset)

In [ ]:
Evaluation_df=pd.DataFrame({
    'Silhouette Score':[Silhouette_score_trainset,Silhouette_score_testset],
    'Davis Bouldin Index': [davies_bouldin_score_trainset,davies_bouldin_score_testset],
    'Calinski Harabasz Score':[calinski_harabasz_score_trainset,calinski_harabasz_score_testset]
},
index=['Gaussian Mixture Model (Train set)','Gaussian Mixture Model (Test set)'])

In [ ]:
Evaluation_df

#### Train Set Evaluation

In [ ]:
pred_failure_train=322
false_failure_train=7
pred_good_train=776
incor_good_preds_train=1

In [ ]:
True_negative_train=322-7+1

In [ ]:
True_positive_train=776-1+7
True_positive_train

In [ ]:
Precision_Gmm_train_good=(True_positive_train)/(True_positive_train+incor_good_preds_train)
Precision_Gmm_train_good

In [ ]:
recall_Gmm_train_good=(True_positive_train)/(True_positive_train+false_failure_train)
recall_Gmm_train_good

In [ ]:
F1_Gmm_train_good=(2*Precision_Gmm_train_good*recall_Gmm_train_good)/(Precision_Gmm_train_good+recall_Gmm_train_good)
F1_Gmm_train_good

In [ ]:
Precision_Gmm_train_failure=(True_negative_train)/(True_negative_train+incor_good_preds_train)
Precision_Gmm_train_failure

In [ ]:
recall_Gmm_train_failure=(True_negative_train)/(True_negative_train+false_failure_train)
recall_Gmm_train_failure

In [ ]:
F1_Gmm_train_failure=(2*Precision_Gmm_train_failure*recall_Gmm_train_failure)/(Precision_Gmm_train_failure+recall_Gmm_train_failure)
F1_Gmm_train_failure

In [ ]:
Evaluation_df_gmm_train=pd.DataFrame({
    'Precision':[Precision_Gmm_train_good,Precision_Gmm_train_failure],
    'Recall': [recall_Gmm_train_good,recall_Gmm_train_failure],
    'F1-Score':[F1_Gmm_train_good,F1_Gmm_train_failure],
},
index=['Good Condition Train set (GMM)','Failure Train set (GMM)'])

In [ ]:
Evaluation_df_gmm_train

In [ ]:
###Accuracy

In [ ]:
Accuracy_Gmm_train=(True_positive_train+True_negative_train)/(True_positive_train+false_failure_train+True_negative_train+incor_good_preds_train)
Accuracy_Gmm_train

In [ ]:
Specificity_Gmm_train=(True_negative_train)/(True_negative_train+incor_good_preds_train)
Specificity_Gmm_train

#### Test Set Evaluation

In [ ]:
pred_failure_test=83
false_failure_test=2
pred_good_test=191
incor_good_preds_test=1

In [ ]:
True_negative_test=83-2+1
True_negative_test

In [ ]:
True_positive_test=191-1+2
True_positive_test

In [ ]:
Precision_Gmm_test_good=(True_positive_test)/(True_positive_test+incor_good_preds_test)
Precision_Gmm_test_good

In [ ]:
recall_Gmm_test_good=(True_positive_test)/(True_positive_test+false_failure_test)
recall_Gmm_test_good

In [ ]:
F1_Gmm_test_good=(2*Precision_Gmm_test_good*recall_Gmm_test_good)/(Precision_Gmm_test_good+recall_Gmm_test_good)
F1_Gmm_test_good

In [ ]:
Precision_Gmm_test_failure=(True_negative_test)/(True_negative_test+incor_good_preds_test)
Precision_Gmm_test_failure

In [ ]:
recall_Gmm_test_failure=(True_negative_test)/(True_negative_test+false_failure_test)
recall_Gmm_test_failure

In [ ]:
F1_Gmm_test_failure=(2*Precision_Gmm_test_failure*recall_Gmm_test_failure)/(Precision_Gmm_test_failure+recall_Gmm_test_failure)
F1_Gmm_test_failure

In [ ]:
Evaluation_df_gmm_test=pd.DataFrame({
    'Precision':[Precision_Gmm_test_good,Precision_Gmm_test_failure],
    'Recall': [recall_Gmm_test_good,recall_Gmm_test_failure],
    'F1-Score':[F1_Gmm_test_good,F1_Gmm_test_failure],
},
index=['Good Condition Test set (GMM)','Failure Test set (GMM)'])

In [ ]:
Evaluation_df_gmm_test

In [ ]:
final_evaluation=pd.concat([Evaluation_df_gmm_train,Evaluation_df_gmm_test],axis=0)

In [ ]:
final_evaluation

In [ ]:
Accuracy_Gmm_test=(True_positive_test+True_negative_test)/(True_positive_test+false_failure_test+True_negative_test+incor_good_preds_test)
Accuracy_Gmm_test

In [ ]:
Specificity_Gmm_test=(True_negative_test)/(True_negative_test+incor_good_preds_test)
Specificity_Gmm_test